# 深度学习计算

## 知识点
- 神经网络：层（layer）、块（block）、网络（net）
- 块需要实现的功能
  1. 将输入作为参数
  2. 通过前向传播生成输出
  3. （自动）计算梯度
  4. 存储和访问前向传播计算所需的参数
  5. 根据需要初始化参数

---

In [2]:
import torch
from torch import nn
from torch.nn import functional as F # 为了使用ReLU的函数版本

net = nn.Sequential(nn.Linear(20, 256), # hidden layer
                    nn.ReLU(),
                    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.0127,  0.0080,  0.0428, -0.0104,  0.0381, -0.1383, -0.0558,  0.2224,
          0.0835,  0.0304],
        [-0.0403,  0.0169,  0.1192,  0.0960,  0.1420, -0.2902,  0.0540,  0.0586,
         -0.0398, -0.1674]], grad_fn=<AddmmBackward0>)

In [5]:
# 自定义块，以Class形式实现
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    def forward(self, X):
        # nn.functional中定义了ReLU的函数版本
        return self.out(F.relu(self.hidden(X)))
    

In [6]:
net = MLP()
net(X)

tensor([[-0.1708,  0.2131, -0.0437, -0.0065, -0.0038, -0.1779, -0.2833, -0.0734,
         -0.1753,  0.1082],
        [-0.1263,  0.1843,  0.0485, -0.0032,  0.1410, -0.1046, -0.2625,  0.1533,
         -0.1458,  0.0413]], grad_fn=<AddmmBackward0>)

In [7]:
'''顺序块'''
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # _modules是nn.Module类自带的一个有序字典
            self._modules[str(idx)] = module
        
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256),
                   nn.ReLU(),
                   nn.Linear(256, 10))
net(X)

tensor([[-2.4551e-01, -7.5283e-02,  2.8649e-01, -4.5333e-02,  7.8900e-03,
         -9.7985e-02,  2.8343e-01,  1.6903e-02,  2.2674e-02, -2.7759e-02],
        [-1.2736e-01, -2.6037e-02,  1.8560e-01, -5.9626e-03,  7.2054e-02,
         -1.5189e-04,  7.6742e-02,  8.9185e-02, -8.0320e-03, -3.3862e-02]],
       grad_fn=<AddmmBackward0>)

In [9]:
# 合并`常数参数`
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 常数参数，不计算梯度
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 过一个自定义的常数层
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，相当于2层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.2991, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64),
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),
                        nn.Linear(16, 20),
                        FixedHiddenMLP())
chimera(X)

tensor(-0.1234, grad_fn=<SumBackward0>)

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8),
                    nn.ReLU(),
                    nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1011],
        [ 0.0561]], grad_fn=<AddmmBackward0>)

In [13]:
from pprint import *
pprint(net[2].state_dict())
# 检查第二个全连接层的参数

OrderedDict([('weight',
              tensor([[ 0.0669,  0.2399,  0.3274,  0.0811,  0.0508, -0.0893,  0.1784, -0.0955]])),
             ('bias', tensor([-0.2728]))])


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2728], requires_grad=True)
tensor([-0.2728])


In [15]:
net[2].weight.grad == None
# 因为还没有backward()，所以梯度尚未被计算

True

In [16]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.2728])

In [18]:
# 工厂方法
def block1():
    return nn.Sequential(nn.Linear(4, 8),
                         nn.ReLU(),
                         nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # add_module：名称（str）、模块对象（此处调用工厂方法）
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(),
                      nn.Linear(4, 1))

rgnet(X)

tensor([[-0.3428],
        [-0.3428]], grad_fn=<AddmmBackward0>)

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [20]:
rgnet[0][1][0].bias.data

tensor([ 0.3958, -0.0090, -0.1034,  0.4001, -0.3735,  0.0590,  0.4348,  0.0700])

In [21]:
'''参数初始化'''
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0077,  0.0146,  0.0106, -0.0118]), tensor(0.))

In [22]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [23]:
# Xaiver初始化方法
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

def init42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
    
net[0].apply(init_xavier)
net[2].apply(init42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1802, -0.6425,  0.4938, -0.6866])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [24]:
'''自定义初始化'''
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",
              *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 分段函数，绝对值大于等于5的保留，小于5的直接清零
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]
        

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.0962,  7.0934,  9.8901, -5.3660],
        [-0.0000, -0.0000,  7.6482,  0.0000]], grad_fn=<SliceBackward0>)

In [25]:
# 直接设置参数
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.0934, 10.8901, -4.3660])

In [26]:
'''参数绑定'''
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1)
                    )
net(X)

tensor([[0.0805],
        [0.0845]], grad_fn=<AddmmBackward0>)

In [28]:
print(net)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)
tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [1]:
'''自定义层'''
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [2]:
net = nn.Sequential(
    nn.Linear(8, 128),
    CenteredLayer()
)
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-10, grad_fn=<MeanBackward0>)

In [3]:
'''自定义Linear层'''
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,)) # 逗号不是必须，但是是一个创建张量的好习惯
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [4]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.2851,  0.3130, -1.0535],
        [ 0.3744, -0.7067,  0.6628],
        [-0.7683,  1.0193, -1.8814],
        [-1.4373,  0.1130, -0.7048],
        [ 1.5829,  0.9937,  0.1744]], requires_grad=True)

In [5]:
'''读写文件'''
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
# 对于单个张量：save
torch.save(x, 'x-file')

In [7]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [8]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [13]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0038, -0.1244,  0.0720,  ..., -0.2091,  0.1744, -0.1317],
                      [-0.2067,  0.0480,  0.2023,  ..., -0.0193, -0.1142, -0.1864],
                      [-0.1939, -0.1459,  0.0196,  ...,  0.2049,  0.1690,  0.1550],
                      ...,
                      [-0.1415,  0.0658, -0.0154,  ..., -0.0614, -0.0993,  0.1658],
                      [ 0.0969,  0.1441, -0.0772,  ..., -0.1971, -0.0265,  0.1917],
                      [-0.0602,  0.2022,  0.0953,  ..., -0.0402,  0.0089,  0.0986]])),
             ('hidden.bias',
              tensor([ 0.1378, -0.1090,  0.1111,  0.0065, -0.1445, -0.1723,  0.1146,  0.1443,
                      -0.0309, -0.0150, -0.0393, -0.0441,  0.0330,  0.1356, -0.1984, -0.1292,
                      -0.0090,  0.2213, -0.2185,  0.0129,  0.1809, -0.0374, -0.0728,  0.1765,
                      -0.0972, -0.0130, -0.0594,  0.1887,  0.1124,  0.1154, -0.0978,  0.1789,
                       0.0132,

In [10]:
torch.save(net.state_dict(), './data/mlp.params')

In [11]:
clone = MLP()
clone.load_state_dict(torch.load('./data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [12]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [20]:
'''使用GPU / CUDA'''
import torch
import torch as nn

torch.device('cpu'), torch.device('cuda')

(device(type='cpu'), device(type='cuda'))

In [4]:
!nvidia-smi

Wed Mar 27 09:42:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:98:00.0 Off |                  Off |
| 30%   31C    P8              22W / 450W |      0MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
torch.cuda.device_count()
# 查询可用GPU数量

NameError: name 'torch' is not defined

In [6]:
def try_gpu(i=0): #@save
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

In [7]:
device = torch.device('mps')
# 惰性，不会立刻验证可用性

In [8]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [10]:
'''将张量存储在GPU上'''
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [11]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8138, 0.0645, 0.5074],
        [0.8717, 0.7009, 0.8911]], device='cuda:1')

In [14]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [16]:
Y + Z

tensor([[1.8138, 1.0645, 1.5074],
        [1.8717, 1.7009, 1.8911]], device='cuda:1')

In [17]:
Z.cuda(1) is Z

True

In [22]:
from torch import nn
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [23]:
net(X)

tensor([[-0.3535],
        [-0.3535]], device='cuda:0', grad_fn=<AddmmBackward0>)